In [20]:
import pandas as pd
import numpy as np
import os
from shapely.geometry import Point, Polygon
import geopandas as gpd
from shapely.ops import nearest_points
import pyproj


cwd =os.getcwd()
#abc

### Structure socioeconomic data by year

In [22]:
socioeco_df = pd.read_excel(cwd+'/socioeconomic_data/socioeco_data.xlsx', sheet_name = 'Samlet')

postnr_columns =['POSTNR', 'YEAR', 'KOMKODE', 'SALGSPRIS']
kom_columns = ['KOMKODE', 'YEAR', 'DISINDK', 'KRIMINELITET', 'LAVINDK', 'LANGLEDIG','SKILSMISSER', 'ANDEL_INDV']

post_df = socioeco_df[postnr_columns]
kom_df = socioeco_df[kom_columns]

years = [2011, 2015, 2019]
post_split = [post_df[post_df['YEAR'] == year] for year in years]
kom_split = [kom_df[kom_df['YEAR'] == year] for year in years]

### Merge school data and socioeconomic data by postal code and municipality

In [23]:
df_list = []
for data, year in zip(post_split, [2011, 2015, 2019]):
    # load data
    soc_data = data
    school_data = pd.read_csv(cwd+f'/school_data/school_clean_{year-1}-{year}.csv', index_col=0)

    # merge data
    merged = pd.merge(left = school_data, right = soc_data, left_on = ['beliggenhedskommunenr', 'postnr'], right_on = ['KOMKODE', 'POSTNR'])

    # format
    merged.columns = merged.columns.str.lower()
    merged = merged.T.drop_duplicates().T

    merged.to_csv(f'postnr_background_vars{year}.csv')

    df_list.append(merged)

for data, year in zip(kom_split, [2011, 2015, 2019]):
    data.drop_duplicates()\
        .to_csv(f'kom_background_vars{year}.csv')

### Gather data for previous election

In [24]:
# load municipality codes and format for merge
kom_koder = pd.read_excel(cwd+'/school_data/data_download/kom_koder.xls', header = 3, usecols = 'G:H')
kom_koder.columns = ['kom_kode', 'kom_navn']
kom_koder['kom_navn'] = kom_koder['kom_navn'] + ' Kommune'
kom_koder = kom_koder[kom_koder['kom_kode'] != 411] # delete Christiansø

In [25]:
# define left and right parties
left_parties = ['Socialdemokratiet', 'Radikale Venstre', 'SF - Socialistisk Folkeparti', 'Enhedslisten - De Rød-Grønne', 'Alternativet']
right_parties = ['Det Konservative Folkeparti', 'Nye Borgerlige', 'Klaus Riskær Pedersen', 'Liberal Alliance', 'Kristendemokraterne', 'Dansk Folkeparti', 'Stram Kurs', 'Venstre, Danmarks Liberale Parti']

for year in [2011, 2015]:
    votes = pd.read_csv(cwd+f'/scrape_geodata/data{year}.csv')

    # construct variables for left and right votes
    left_dummy = [int(party in left_parties) for party in votes['party']]
    right_dummy = [int(party in right_parties) for party in votes['party']]
    independent_dummy = [int(party not in left_parties+right_parties) for party in votes['party']]


    votes['left_dummy'], votes['right_dummy'], votes['independent_dummy'] = left_dummy, right_dummy, independent_dummy
    votes['left_votes'], votes['right_votes'], votes['independent_votes'] = left_dummy*votes['votes'], right_dummy*votes['votes'], independent_dummy*votes['votes']

    # format for 2011
    if year == 2011:
        votes = votes[['municipality_x', 'left_votes', 'right_votes']]\
                    .groupby(by = 'municipality_x')\
                    .sum()
        votes = votes.reset_index()\
                    .rename(columns = {'municipality_x': 'municipality'})
    else:
        votes = votes[['municipality', 'left_votes', 'right_votes']].groupby(by = 'municipality').sum()

    # construct vote shares
    votes['left_share'] = votes['left_votes']/(votes['left_votes']+votes['right_votes'])
    votes['right_share'] = votes['right_votes']/(votes['left_votes']+votes['right_votes'])

    # merge to kom_koder
    votes = pd.merge(left = votes, right = kom_koder, left_on = 'municipality', right_on = 'kom_navn').drop('kom_navn', axis = 1)

    votes.to_csv(f'histvotes{year}.csv', encoding = 'utf-8-sig')

### Merge to closest school within same municipality, merge all data together and save as 

In [27]:
year = 2019

# Load municipality codes
kom_koder = list(pd.read_excel(cwd+'/school_data/data_download/kom_koder.xls', usecols = 'G:H', header = 3).iloc[:,0])
kom_koder.remove(411) # delete Christiansø

# Project geographical background variales from lat-lon to utm
df = pd.read_csv('postnr_background_vars'+str(year)+'.csv', index_col=0)
c = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry = c)

gdf.crs = {'init': 'epsg:4326'}
gdf_utm = gdf.to_crs(epsg = 32632)

# Narrow down to primary schools only
gdf_utm = gdf_utm[gdf_utm['institutionstype2'] == 'Grundskoler']


# Load election data and find centroid for every district
valg_distrikt = gpd.read_file(cwd+'\scrape_geodata\geodata\dagi_10m_nohist_l1.afstemningsomraade\\afstemningsomraade.shp', driver = 'ESRI Shapefile')
valg_distrikt['centroid']=valg_distrikt['geometry'].centroid
valg_geo = valg_distrikt.set_geometry('centroid')

# Split into list of backgrounddata and election data by municipality code
kommune_split = [[gdf_utm[gdf_utm['beliggenhedskommunenr'] == kom_kode], valg_geo[valg_geo['kommunekod'].astype(int) == kom_kode]] for kom_kode in kom_koder]

# For each municipality, find closest school to each polling place.
def near(point, pts):
    """
    Finds closest among pts to point and saves the relevant institutionsnr
    """
    nearest = gdf_utm.geometry == nearest_points(point, pts)[1]
    return gdf_utm[nearest]['institutionsnummer'].values[0]

for gdf, valg_geo in kommune_split:
    pts = gdf.geometry.unary_union
    valg_geo['institutionsnummer'] = valg_geo.apply(lambda row: near(row.geometry, pts), axis = 1)



# concat school data
gdf_utm = pd.concat([kommune_split[i][0] for i in range(len(kommune_split))])

# concat election data
valg_geo = pd.concat([kommune_split[i][1] for i in range(len(kommune_split))])

# merge election to school in 'intitutionsnummer
merge1 = pd.merge(left=valg_geo, right = gdf_utm, on = 'institutionsnummer')


### CHECKPOINT ###

# merge merge1 to mapping on 'objectid'
mapping = pd.read_excel(cwd+f'/scrape_geodata/mapping{year}.xlsx')
mapping.objectid = mapping.objectid.astype(str)
merge2 = pd.merge(left = merge1, right = mapping[['objectid', 'refs']], on = 'objectid')

# merge to election results
election_result = pd.read_csv(cwd+f'/scrape_geodata/data{year}.csv')

merge3 = pd.merge(left = merge2, right = election_result, on = 'refs')
merge3.kommunekod = merge3.kommunekod.astype(int).astype(str)

# merge to background vars on kommune level
background = pd.read_csv(f'kom_background_vars{year}.csv', index_col = 0)

background.KOMKODE = background.KOMKODE.astype(str)
background

all_ = pd.merge(left = merge3, right = background, left_on = 'kommunekod', right_on = 'KOMKODE')

all_.columns = all_.columns.str.lower()
col_vars = ['objectid','navn_x', 'afstemni00','kommunekod', 'institutionsnummer', 'karakter', 'soc_ref', 'postnr', 'navn_y', 'institutionstype2', 'institutionstype3', 'year_x', 'salgspris', 'party', 'votes', 'eligible_count', 'blank', 'invalid', 'total_valid', 'total_casted', 'time_stamp', 'names', 'constituency', 'voting_area', 'municipality', 'disindk', 'kriminelitet', 'lavindk', 'langledig', 'skilsmisser', 'andel_indv', 'geometry_x']

temp_ = all_[col_vars]


# create variables for vote counts etc
left_parties = ['Socialdemokratiet', 'Radikale Venstre', 'SF - Socialistisk Folkeparti', 'Enhedslisten - De Rød-Grønne', 'Alternativet']
right_parties = ['Det Konservative Folkeparti', 'Nye Borgerlige', 'Klaus Riskær Pedersen', 'Liberal Alliance', 'Kristendemokraterne', 'Dansk Folkeparti', 'Stram Kurs', 'Venstre, Danmarks Liberale Parti']

left_dummy = [int(party in left_parties) for party in temp_['party']]
right_dummy = [int(party in right_parties) for party in temp_['party']]
independent_dummy = [int(party not in left_parties+right_parties) for party in temp_['party']]


temp_['left_dummy'], temp_['right_dummy'], temp_['independent_dummy'] = left_dummy, right_dummy, independent_dummy

temp_['left_votes'], temp_['right_votes'], temp_['independent_votes'] = left_dummy*temp_['votes'], right_dummy*temp_['votes'], independent_dummy*temp_['votes']

votes_df = temp_[['objectid', 'votes', 'left_votes', 'right_votes', 'independent_votes']].groupby(by = 'objectid').sum()

pre_merge = temp_.drop(['votes','left_votes', 'right_votes', 'independent_votes', 'left_dummy', 'right_dummy', 'independent_dummy', 'party'], axis = 1)
finalish_df = pd.merge(left = pre_merge, right = votes_df, on = 'objectid')\
            .drop_duplicates(subset = 'objectid')\
            .reset_index(drop = True)

finalish_df['left_share'] = finalish_df['left_votes']/finalish_df['total_valid']
finalish_df['right_share'] = finalish_df['right_votes']/finalish_df['total_valid']
finalish_df['independent_share'] = finalish_df['independent_votes']/finalish_df['total_valid']
finalish_df['karakter_forskel'] = finalish_df['soc_ref']-finalish_df['karakter']

final_gdf = gpd.GeoDataFrame(finalish_df[['objectid', 'afstemni00','navn_x', 'kommunekod', 'municipality', 'constituency','postnr', 'time_stamp', 'votes', 'eligible_count', 'total_casted', 'total_valid', 'left_votes', 'right_votes', 'independent_votes', 'blank', 'invalid', 'left_share', 'right_share', 'independent_share', 'institutionsnummer', 'navn_y', 'institutionstype2', 'institutionstype3', 'karakter', 'soc_ref', 'karakter_forskel', 'salgspris', 'disindk', 'kriminelitet', 'lavindk', 'langledig','skilsmisser', 'andel_indv', 'geometry_x']])

final_gdf = final_gdf.rename(columns = {'navn_x': 'afstem_navn2', 'afstemni00': 'afstem_navn1', 'soc_ref': 'karakter_soc_ref', 'navn_y': 'skole_navn', 'blank': 'blank_votes', 'invalid': 'invalid_votes', 'votes': 'votes', 'municipality': 'kommunenavn', 'constituency': 'opst_kreds', 'geometry_x':'geometry'})

final_gdf['area'] = final_gdf.geometry.area
final_gdf['pop_density'] = final_gdf['eligible_count']/final_gdf['area']


# add historical votes
histvotes = pd.read_csv(f'histvotes{year-4}.csv', index_col = 0)
histvotes = histvotes.rename(columns = {'left_share': 'hist_left_share', 'right_share': 'hist_right_share'})\
            [['hist_left_share', 'hist_right_share', 'kom_kode']]
histvotes['kom_kode'] = histvotes['kom_kode'].astype(str)

final_gdf = pd.merge(left = final_gdf, right = histvotes, left_on = 'kommunekod', right_on = 'kom_kode').drop('kom_kode', axis = 1)

final_gdf.to_csv(f'final_data{year}.csv', encoding = 'utf-8-sig')

# save geometry
geometry = final_gdf.geometry
geometry.to_file(f'geometry{year}.shp')



In [11]:
# Load municipality codes
kom_koder = list(pd.read_excel(cwd+'/school_data/data_download/kom_koder.xls', usecols = 'G:H', header = 3).iloc[:,0])
kom_koder.remove(411) # delete Christiansø


# 2019 works

for year in [2015, 2019]:
    # Project geographical background variales from lat-lon to utm
    df = pd.read_csv('postnr_background_vars'+str(year)+'.csv', index_col=0)
    c = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]
    gdf = gpd.GeoDataFrame(df, geometry = c)

    gdf.crs = {'init': 'epsg:4326'}
    gdf_utm = gdf.to_crs(epsg = 32632)

    # Narrow down to primary schools only
    gdf_utm = gdf_utm[gdf_utm['institutionstype2'] == 'Grundskoler']


    # Load election data and find centroid for every district
    valg_distrikt = gpd.read_file(cwd+'\scrape_geodata\geodata\dagi_10m_nohist_l1.afstemningsomraade\\afstemningsomraade.shp', driver = 'ESRI Shapefile')
    valg_distrikt['centroid']=valg_distrikt['geometry'].centroid
    valg_geo = valg_distrikt.set_geometry('centroid')

    # Split into list of backgrounddata and election data by municipality code
    kommune_split = [[gdf_utm[gdf_utm['beliggenhedskommunenr'] == kom_kode], valg_geo[valg_geo['kommunekod'].astype(int) == kom_kode]] for kom_kode in kom_koder]

    # For each municipality, find closest school to each polling place.
    def near(point, pts):
        """
        Finds closest among pts to point and saves the relevant institutionsnr
        """
        nearest = gdf_utm.geometry == nearest_points(point, pts)[1]
        return gdf_utm[nearest]['institutionsnummer'].values[0]

    for gdf, valg_geo in kommune_split:
        pts = gdf.geometry.unary_union
        valg_geo['institutionsnummer'] = valg_geo.apply(lambda row: near(row.geometry, pts), axis = 1)



    # concat school data
    gdf_utm = pd.concat([kommune_split[i][0] for i in range(len(kommune_split))])

    # concat election data
    valg_geo = pd.concat([kommune_split[i][1] for i in range(len(kommune_split))])

    # merge election to school in 'intitutionsnummer
    merge1 = pd.merge(left=valg_geo, right = gdf_utm, on = 'institutionsnummer')


    ### CHECKPOINT ###

    # merge merge1 to mapping on 'objectid'
    mapping = pd.read_excel(cwd+f'/scrape_geodata/mapping{year}.xlsx')
    mapping.objectid = mapping.objectid.astype(str)
    merge2 = pd.merge(left = merge1, right = mapping[['objectid', 'refs']], on = 'objectid')

    # merge to election results
    election_result = pd.read_csv(cwd+f'/scrape_geodata/data{year}.csv')

    merge3 = pd.merge(left = merge2, right = election_result, on = 'refs')
    merge3.kommunekod = merge3.kommunekod.astype(int).astype(str)

    # merge to background vars on kommune level
    background = pd.read_csv(f'kom_background_vars{year}.csv', index_col = 0)

    background.KOMKODE = background.KOMKODE.astype(str)
    background

    all_ = pd.merge(left = merge3, right = background, left_on = 'kommunekod', right_on = 'KOMKODE')

    all_.columns = all_.columns.str.lower()
    col_vars = ['objectid','navn_x', 'afstemni00','kommunekod', 'institutionsnummer', 'karakter', 'soc_ref', 'postnr', 'navn_y', 'institutionstype2', 'institutionstype3', 'year_x', 'salgspris', 'party', 'votes', 'eligible_count', 'blank', 'invalid', 'total_valid', 'total_casted', 'time_stamp', 'names', 'constituency', 'voting_area', 'municipality', 'disindk', 'kriminelitet', 'lavindk', 'langledig', 'skilsmisser', 'andel_indv', 'geometry_x']

    temp_ = all_[col_vars]


    # create variables for vote counts etc
    left_parties = ['Socialdemokratiet', 'Radikale Venstre', 'SF - Socialistisk Folkeparti', 'Enhedslisten - De Rød-Grønne', 'Alternativet']
    right_parties = ['Det Konservative Folkeparti', 'Nye Borgerlige', 'Klaus Riskær Pedersen', 'Liberal Alliance', 'Kristendemokraterne', 'Dansk Folkeparti', 'Stram Kurs', 'Venstre, Danmarks Liberale Parti']

    left_dummy = [int(party in left_parties) for party in temp_['party']]
    right_dummy = [int(party in right_parties) for party in temp_['party']]
    independent_dummy = [int(party not in left_parties+right_parties) for party in temp_['party']]


    temp_['left_dummy'], temp_['right_dummy'], temp_['independent_dummy'] = left_dummy, right_dummy, independent_dummy

    temp_['left_votes'], temp_['right_votes'], temp_['independent_votes'] = left_dummy*temp_['votes'], right_dummy*temp_['votes'], independent_dummy*temp_['votes']

    votes_df = temp_[['objectid', 'votes', 'left_votes', 'right_votes', 'independent_votes']].groupby(by = 'objectid').sum()

    pre_merge = temp_.drop(['votes','left_votes', 'right_votes', 'independent_votes', 'left_dummy', 'right_dummy', 'independent_dummy', 'party'], axis = 1)
    finalish_df = pd.merge(left = pre_merge, right = votes_df, on = 'objectid')\
                .drop_duplicates(subset = 'objectid')\
                .reset_index(drop = True)

    finalish_df['left_share'] = finalish_df['left_votes']/finalish_df['total_valid']
    finalish_df['right_share'] = finalish_df['right_votes']/finalish_df['total_valid']
    finalish_df['independent_share'] = finalish_df['independent_votes']/finalish_df['total_valid']
    finalish_df['karakter_forskel'] = finalish_df['soc_ref']-finalish_df['karakter']

    final_gdf = gpd.GeoDataFrame(finalish_df[['objectid', 'afstemni00','navn_x', 'kommunekod', 'municipality', 'constituency','postnr', 'time_stamp', 'votes', 'eligible_count', 'total_casted', 'total_valid', 'left_votes', 'right_votes', 'independent_votes', 'blank', 'invalid', 'left_share', 'right_share', 'independent_share', 'institutionsnummer', 'navn_y', 'institutionstype2', 'institutionstype3', 'karakter', 'soc_ref', 'karakter_forskel', 'salgspris', 'disindk', 'kriminelitet', 'lavindk', 'langledig','skilsmisser', 'andel_indv', 'geometry_x']])

    final_gdf = final_gdf.rename(columns = {'navn_x': 'afstem_navn2', 'afstemni00': 'afstem_navn1', 'soc_ref': 'karakter_soc_ref', 'navn_y': 'skole_navn', 'blank': 'blank_votes', 'invalid': 'invalid_votes', 'votes': 'votes', 'municipality': 'kommunenavn', 'constituency': 'opst_kreds', 'geometry_x':'geometry'})

    final_gdf['area'] = final_gdf.geometry.area
    final_gdf['pop_density'] = final_gdf['eligible_count']/final_gdf['area']


    # add historical votes
    histvotes = pd.read_csv(f'histvotes{year-4}.csv', index_col = 0)
    histvotes = histvotes.rename(columns = {'left_share': 'hist_left_share', 'right_share': 'hist_right_share'})\
                [['hist_left_share', 'hist_right_share', 'kom_kode']]
    histvotes['kom_kode'] = histvotes['kom_kode'].astype(str)

    final_gdf = pd.merge(left = final_gdf, right = histvotes, left_on = 'kommunekod', right_on = 'kom_kode').drop('kom_kode', axis = 1)

    final_gdf.to_csv(f'final_data{year}.csv', encoding = 'utf-8-sig')

    # save geometry
    geometry = final_gdf.geometry
    geometry.to_file(f'geometry{year}.shp')



ValueError: The second input geometry is empty